In [1]:
import os
import json
import boto3
import sklearn
import socket
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
#from pyspark.sql import types as T

print('user:', os.environ['JUPYTERHUB_SERVICE_PREFIX'])

def uiWebUrl(self):
    from urllib.parse import urlparse
    web_url = self._jsc.sc().uiWebUrl().get()
    port = urlparse(web_url).port
    return "{}proxy/{}/jobs/".format(os.environ['JUPYTERHUB_SERVICE_PREFIX'], port)

# small fix to enable UI views
SparkContext.uiWebUrl = property(uiWebUrl)

# spark configurtion in local regime 
conf = SparkConf().set('spark.master', 'local[*]').set('spark.driver.memory', '8g')

#some needed objects
sc = SparkContext(conf=conf)
spark = SparkSession(sc)
spark

user: /user/st054328/


# Homework 1

Transform text file "The Project Gutenberg eBook of Frankenstein, by Mary Wollstonecraft (Godwin) Shelle" into TF-IDF. Take sentence as "document".

### Part 1: 
- read text file as dataframe 
- filter out non-letters and empty strings 
- transform into dataframe doc_id -> tf_idf vector 


### Part 2:
- read text file as RDD
- filter out non-letters and empty strings 
- transform into rdd in format doc_id -> tf_idf vector


### Org part: 
I'm waiting your HW's as self-sufficient jupyter notebooks in github repository. 

Please, fill this table in specified comment with:

your name / github link / telegram (optionally, if u want to discuss some) / 

Fill the comment please and i will add your data in a few days

https://docs.google.com/spreadsheets/d/1p3yLsXqX2dp_TrPwNcikcS5FP_PTM0_gnSOzGn5Gn1E/edit#gid=0

Feel free to text me if u have some questions 

### Deadline: 01.05.2021 included

Dear students, dead in "deadline" means *dead*. This deadline is not for you - it's for me. Deadlines informs me from which point i should start to score your HWs.  You can commit anything after deadline but it's not guaranteed that I'll take it into account. It's possible to move deadline only for the whole group not "just for me plz cause I was ill / detentioned / skipped this message". 

### NB(!) 

It's not allowed to use TF-IDF code from Spark internal libraries. 
It's not allowed to cast DF/RDD into pandas and use scikit-learn. Please, keep it spark. 


## Part 1

Reading text file as dataframe

In [2]:
result_prefix = "malyutin_demo_hw1"

filepath = "file:///home/jovyan/shared/lectures_folder/84-0.txt"
from pyspark.sql.functions import monotonically_increasing_id

dataframe = sc.textFile(f"{filepath}")\
    .map(lambda x: (x,))\
    .toDF()\
    .select(F.col("_1").alias("text"))\
    .withColumn("id", monotonically_increasing_id())

dataframe.show()

+--------------------+---+
|                text| id|
+--------------------+---+
|The Project Guten...|  0|
|                    |  1|
|This eBook is for...|  2|
|most other parts ...|  3|
|whatsoever. You m...|  4|
|of the Project Gu...|  5|
|www.gutenberg.org...|  6|
|will have to chec...|  7|
|   using this eBook.|  8|
|                    |  9|
| Title: Frankenstein| 10|
|       or, The Mo...| 11|
|                    | 12|
|Author: Mary Woll...| 13|
|                    | 14|
|Release Date: 31,...| 15|
|[Most recently up...| 16|
|                    | 17|
|   Language: English| 18|
|                    | 19|
+--------------------+---+
only showing top 20 rows



In [3]:
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType, MapType
from pyspark.mllib.linalg import Vectors, DenseVector

import string
import re
import math
import numpy as np

def process_string(data):
    """
    basic preprocessing function:
    - removes punctuation
    - lower
    - split by space
    """
    punct_removed = re.sub(r'[^a-zA-Z ]','',data)
    words = punct_removed.lower().split(" ")
    
    
    return list(filter(lambda x: len(x) > 0, words))

# spark udf -- user defined function (~ mapper)

process_string_udf = udf(lambda z: process_string(z), ArrayType(StringType()))

In [4]:
"""
process words

filter out empty and small sentences
"""

by_words = dataframe\
    .select(process_string_udf(F.col("text")).alias("text"), "id")\
    .where(F.size(F.col("text")) >= 1)#\
    #.withColumn("id", monotonically_increasing_id())


by_words.show()

+--------------------+---+
|                text| id|
+--------------------+---+
|[the, project, gu...|  0|
|[this, ebook, is,...|  2|
|[most, other, par...|  3|
|[whatsoever, you,...|  4|
|[of, the, project...|  5|
|[wwwgutenbergorg,...|  6|
|[will, have, to, ...|  7|
|[using, this, ebook]|  8|
|[title, frankenst...| 10|
|[or, the, modern,...| 11|
|[author, mary, wo...| 13|
|[release, date, e...| 15|
|[most, recently, ...| 16|
| [language, english]| 18|
|[character, set, ...| 20|
|[produced, by, ju...| 22|
|[further, correct...| 23|
|[start, of, the, ...| 25|
|      [frankenstein]| 30|
|[or, the, modern,...| 32|
+--------------------+---+
only showing top 20 rows



Let's explode the words in each text.

In [5]:
unfolded = by_words\
.select(by_words.text.alias('document'), by_words.id.alias('doc_id'), F.explode(by_words.text).alias('token'))\
.withColumn("id", monotonically_increasing_id())

unfolded.show()

+--------------------+------+--------------+---+
|            document|doc_id|         token| id|
+--------------------+------+--------------+---+
|[the, project, gu...|     0|           the|  0|
|[the, project, gu...|     0|       project|  1|
|[the, project, gu...|     0|     gutenberg|  2|
|[the, project, gu...|     0|         ebook|  3|
|[the, project, gu...|     0|            of|  4|
|[the, project, gu...|     0|  frankenstein|  5|
|[the, project, gu...|     0|            by|  6|
|[the, project, gu...|     0|          mary|  7|
|[the, project, gu...|     0|wollstonecraft|  8|
|[the, project, gu...|     0|        godwin|  9|
|[the, project, gu...|     0|       shelley| 10|
|[this, ebook, is,...|     2|          this| 11|
|[this, ebook, is,...|     2|         ebook| 12|
|[this, ebook, is,...|     2|            is| 13|
|[this, ebook, is,...|     2|           for| 14|
|[this, ebook, is,...|     2|           the| 15|
|[this, ebook, is,...|     2|           use| 16|
|[this, ebook, is,..

Let's count the number of each certain word `t` in the text and the total number of words `n` in each text. Then let's calculate `TF` as $t/n$.

In [6]:
tf0 = unfolded.groupBy("doc_id", "token").agg(F.count("document").alias("t"))

tf = tf0\
.join(tf0.groupBy("doc_id").agg(F.count("doc_id").alias("n")), on="doc_id", how = "left")\
.orderBy("doc_id")\
.withColumn("TF", F.col("t")/F.col("n"))

tf.show()

+------+--------------+---+---+-------------------+
|doc_id|         token|  t|  n|                 TF|
+------+--------------+---+---+-------------------+
|     0|     gutenberg|  1| 11|0.09090909090909091|
|     0|  frankenstein|  1| 11|0.09090909090909091|
|     0|wollstonecraft|  1| 11|0.09090909090909091|
|     0|         ebook|  1| 11|0.09090909090909091|
|     0|       shelley|  1| 11|0.09090909090909091|
|     0|            by|  1| 11|0.09090909090909091|
|     0|        godwin|  1| 11|0.09090909090909091|
|     0|            of|  1| 11|0.09090909090909091|
|     0|       project|  1| 11|0.09090909090909091|
|     0|          mary|  1| 11|0.09090909090909091|
|     0|           the|  1| 11|0.09090909090909091|
|     2|          this|  1| 13|0.07692307692307693|
|     2|        united|  1| 13|0.07692307692307693|
|     2|            of|  1| 13|0.07692307692307693|
|     2|           and|  1| 13|0.07692307692307693|
|     2|      anywhere|  1| 13|0.07692307692307693|
|     2|    

Now let's calculate the numbers of documents `df` containing each word:

In [7]:
df = unfolded.groupBy("token").agg(F.countDistinct("doc_id").alias("df")).orderBy("df", ascending=False)
df.show()

+-----+----+
|token|  df|
+-----+----+
|  the|3282|
|  and|2702|
|   of|2435|
|    i|2356|
|   to|1901|
|   my|1534|
|    a|1311|
|   in|1127|
| that| 971|
|  was| 949|
|   me| 795|
| with| 694|
|  but| 681|
|  had| 649|
|which| 554|
|  you| 550|
|   he| 547|
|   it| 533|
|  not| 519|
|  for| 505|
+-----+----+
only showing top 20 rows



Let's devide the total number of documents `D` by the number of documents containing each word `df`:

In [8]:
d_df = df.withColumn('D/df', unfolded.select(F.countDistinct("doc_id").alias('len')).collect()[0]['len']/df.df)
d_df.show()

+-----+----+------------------+
|token|  df|              D/df|
+-----+----+------------------+
|  the|3282|2.0527117611212677|
|  and|2702| 2.493338267949667|
|   of|2435| 2.766735112936345|
|    i|2356| 2.859507640067912|
|   to|1901| 3.543924250394529|
|   my|1534| 4.391786179921773|
|    a|1311| 5.138825324180015|
|   in|1127| 5.977817213842059|
| that| 971| 6.938208032955716|
|  was| 949| 7.099051633298209|
|   me| 795| 8.474213836477988|
| with| 694| 9.707492795389049|
|  but| 681|   9.8928046989721|
|  had| 649|10.380585516178737|
|which| 554|12.160649819494585|
|  you| 550| 12.24909090909091|
|   he| 547|12.316270566727605|
|   it| 533|12.639774859287055|
|  not| 519|12.980732177263969|
|  for| 505|13.340594059405941|
+-----+----+------------------+
only showing top 20 rows



And let's take 10-based logarithm of `D/df` to get `IDF`:

In [9]:
idf = d_df.withColumn('IDF', F.log(10.0, F.col('D/df')))
idf.show()

+-----+----+------------------+-------------------+
|token|  df|              D/df|                IDF|
+-----+----+------------------+-------------------+
|  the|3282|2.0527117611212677| 0.3123279706356039|
|  and|2702| 2.493338267949667| 0.3967812026666665|
|   of|2435| 2.766735112936345| 0.4419675818020251|
|    i|2356| 2.859507640067912|0.45629126123761427|
|   to|1901| 3.543924250394529| 0.5494844304872352|
|   my|1534| 4.391786179921773| 0.6426411877397161|
|    a|1311| 5.138825324180015|  0.710863855662594|
|   in|1127| 5.977817213842059| 0.7765426313065718|
| that| 971| 6.938208032955716| 0.8412473174446734|
|  was| 949| 7.099051633298209| 0.8512003349253856|
|   me| 795| 8.474213836477988|  0.928099418696208|
| with| 694| 9.707492795389049| 0.9871070768978234|
|  but| 681|   9.8928046989721| 0.9953194354398932|
|  had| 649|10.380585516178737|  1.016221850552309|
|which| 554|12.160649819494585| 1.0849567826242486|
|  you| 550| 12.24909090909091| 1.0881038578584346|
|   he| 547|

To calculate `TF_IDF`, let's multiply `TF` and `IDF`:

In [10]:
tf_idf = tf\
.join(idf, on='token', how='left')\
.orderBy("doc_id")\
.withColumn('TF_IDF', F.col("tf") * F.col("idf"))\
.select("token", "doc_id", "n", "TF_IDF")

tf_idf.show()

+--------------+------+---+--------------------+
|         token|doc_id|  n|              TF_IDF|
+--------------+------+---+--------------------+
|            of|     0| 11|0.040178871072911376|
|wollstonecraft|     0| 11|  0.3046677538757287|
|          mary|     0| 11|  0.3046677538757287|
|        godwin|     0| 11|  0.3046677538757287|
|           the|     0| 11|0.028393451875963992|
|       shelley|     0| 11|  0.3046677538757287|
|       project|     0| 11| 0.17127126138204635|
|         ebook|     0| 11| 0.24677483591325833|
|     gutenberg|     0| 11| 0.21246407759258232|
|            by|     0| 11| 0.10429320999791737|
|  frankenstein|     0| 11| 0.21509714085942927|
|        states|     2| 13| 0.19441819628374596|
|           use|     2| 13|   0.192788250201443|
|            is|     2| 13| 0.10339743907737634|
|            in|     2| 13|0.059734048562043986|
|            of|     2| 13|0.033997506292463474|
|        anyone|     2| 13| 0.22109415676256564|
|           the|    

Let's recover the order of words in text...

In [11]:
tf_idf2 = unfolded.select("id", "doc_id", "token").join(tf_idf, on=["doc_id", "token"], how="left").orderBy("id")
tf_idf2.show()

+------+--------------+---+---+--------------------+
|doc_id|         token| id|  n|              TF_IDF|
+------+--------------+---+---+--------------------+
|     0|           the|  0| 11|0.028393451875963992|
|     0|       project|  1| 11| 0.17127126138204635|
|     0|     gutenberg|  2| 11| 0.21246407759258232|
|     0|         ebook|  3| 11| 0.24677483591325833|
|     0|            of|  4| 11|0.040178871072911376|
|     0|  frankenstein|  5| 11| 0.21509714085942927|
|     0|            by|  6| 11| 0.10429320999791737|
|     0|          mary|  7| 11|  0.3046677538757287|
|     0|wollstonecraft|  8| 11|  0.3046677538757287|
|     0|        godwin|  9| 11|  0.3046677538757287|
|     0|       shelley| 10| 11|  0.3046677538757287|
|     2|          this| 11| 13| 0.09100340600027589|
|     2|         ebook| 12| 13|  0.2088094765419878|
|     2|            is| 13| 13| 0.10339743907737634|
|     2|           for| 14| 13| 0.08655193609492438|
|     2|           the| 15| 13| 0.048050457020

I tried to transform this dataframe into the format doc_id -> tf_idf vector, but didn't succeed.

In [12]:
#from tqdm import tqdm

#tfidfs = []
#for i in tqdm(indicies):
    #df = tf_idf2.where(F.col('doc_id')==i).select("TF_IDF")
    #tfidfs.append([(row.TF_IDF) for row in df.collect()])

So I tried a little bit different approach.

Let's save the IDFs, we've already calculated into the dictionary `dict_dict`.

In [13]:
dict_list = [{row.token : row.IDF} for row in idf.collect()]

dict_dict = {}
for element in dict_list:
    dict_dict.update(element)

Now let's define a function returning a sparse vector with the values of TF-IDFs.

In [14]:
numFeatures = len(dict_dict) #the number of features equals the total number of words in the text

In [15]:
def transform(document):
    freq = {}
    tfidfs = {}
    for term in document:
        i = hash(term) % numFeatures #creating "index" for the word
        freq[i] = freq.get(i, 0) + 1.0 #calculate the number of occurances of each word in a text
        tfidfs[i] = freq[i]/len(document) * dict_dict[term] #calculate TF and multiply it by previously calculated IDF
    return [document, Vectors.sparse(numFeatures, tfidfs.items())] 

In [16]:
#Extracting the texts from previously created dataframe

documents2 = by_words.select('text').rdd.map(list)

def unpack(document):
    texts = []
    for term in document[0]:
        #text = term
        texts.append(term)
    return texts

documents3 = documents2.map(unpack)

Let's apply the function to get TF_IDF sparse vector.

In [17]:
columns = ['text', 'TFIDF_vector']
frequencyVectors = documents3.map(transform)
final = frequencyVectors.toDF(columns).join(by_words, on='text', how='left').orderBy('id').select('id', 'TFIDF_vector')

final.show()

+---+--------------------+
| id|        TFIDF_vector|
+---+--------------------+
|  0|(7435,[285,1364,1...|
|  2|(7435,[140,285,46...|
|  2|(7435,[140,285,46...|
|  3|(7435,[75,285,506...|
|  4|(7435,[632,1204,1...|
|  5|(7435,[285,492,71...|
|  6|(7435,[140,1448,3...|
|  7|(7435,[285,889,12...|
|  8|(7435,[1196,1364,...|
| 10|(7435,[5972,6856]...|
| 11|(7435,[2694,3102,...|
| 11|(7435,[2694,3102,...|
| 13|(7435,[1482,2760,...|
| 15|(7435,[1364,1442,...|
| 16|(7435,[1412,2389,...|
| 18|(7435,[3440,4316]...|
| 20|(7435,[2501,3260,...|
| 22|(7435,[54,1267,13...|
| 23|(7435,[1909,2009,...|
| 25|(7435,[285,1364,3...|
+---+--------------------+
only showing top 20 rows



The sparse TF_IDF vector for the first row looks as follows:

In [18]:
frequencyVectors.take(1)[0][1]

SparseVector(7435, {285: 0.0402, 1364: 0.2468, 1482: 0.3047, 2760: 0.3047, 2941: 0.1043, 3396: 0.2125, 3737: 0.1713, 6357: 0.3047, 6747: 0.0284, 6856: 0.2151, 7280: 0.3047})

## Part 2

Reading text file into RDD

In [19]:
#def clear_line(line):
#    punct_removed = re.sub(r'[^\w\s]','',line)
#    return punct_removed.lower().split(" ")

Let's read texts and add index to each document:

In [20]:
rddText = sc.textFile(f"{filepath}").map(lambda line: process_string(line))#.repartition(1).zipWithIndex().repartition(5)

rdd_with_indicies = rddText.zipWithIndex()
rdd_with_indicies.take(3)

[(['the',
   'project',
   'gutenberg',
   'ebook',
   'of',
   'frankenstein',
   'by',
   'mary',
   'wollstonecraft',
   'godwin',
   'shelley'],
  0),
 ([], 1),
 (['this',
   'ebook',
   'is',
   'for',
   'the',
   'use',
   'of',
   'anyone',
   'anywhere',
   'in',
   'the',
   'united',
   'states',
   'and'],
  2)]

And separatly let's make rdd without empty elements (and with no indexing).

In [21]:
rdd_cleared = rddText.filter(bool)
rdd_cleared.take(3)

[['the',
  'project',
  'gutenberg',
  'ebook',
  'of',
  'frankenstein',
  'by',
  'mary',
  'wollstonecraft',
  'godwin',
  'shelley'],
 ['this',
  'ebook',
  'is',
  'for',
  'the',
  'use',
  'of',
  'anyone',
  'anywhere',
  'in',
  'the',
  'united',
  'states',
  'and'],
 ['most',
  'other',
  'parts',
  'of',
  'the',
  'world',
  'at',
  'no',
  'cost',
  'and',
  'with',
  'almost',
  'no',
  'restrictions']]

To calculate IDFs, let's save the total number of texts into `text_num`:

In [22]:
texts_num = rdd_cleared.count()
texts_num

6737

Now we need to count the number of texts, containing each word. Let's initialize counting by setting 1 to each word.

In [23]:
def counter(document):
    lst = []
    for word in document:
        lst.append((word, 1))
    return lst

In [24]:
count_initializer = rdd_cleared.map(lambda word: counter(word))
count_initializer.take(20)

[[('the', 1),
  ('project', 1),
  ('gutenberg', 1),
  ('ebook', 1),
  ('of', 1),
  ('frankenstein', 1),
  ('by', 1),
  ('mary', 1),
  ('wollstonecraft', 1),
  ('godwin', 1),
  ('shelley', 1)],
 [('this', 1),
  ('ebook', 1),
  ('is', 1),
  ('for', 1),
  ('the', 1),
  ('use', 1),
  ('of', 1),
  ('anyone', 1),
  ('anywhere', 1),
  ('in', 1),
  ('the', 1),
  ('united', 1),
  ('states', 1),
  ('and', 1)],
 [('most', 1),
  ('other', 1),
  ('parts', 1),
  ('of', 1),
  ('the', 1),
  ('world', 1),
  ('at', 1),
  ('no', 1),
  ('cost', 1),
  ('and', 1),
  ('with', 1),
  ('almost', 1),
  ('no', 1),
  ('restrictions', 1)],
 [('whatsoever', 1),
  ('you', 1),
  ('may', 1),
  ('copy', 1),
  ('it', 1),
  ('give', 1),
  ('it', 1),
  ('away', 1),
  ('or', 1),
  ('reuse', 1),
  ('it', 1),
  ('under', 1),
  ('the', 1),
  ('terms', 1)],
 [('of', 1),
  ('the', 1),
  ('project', 1),
  ('gutenberg', 1),
  ('license', 1),
  ('included', 1),
  ('with', 1),
  ('this', 1),
  ('ebook', 1),
  ('or', 1),
  ('online',

Now if we remove duplicates in each text and then sum up values with the same keys, we will get exactly the number of documents, containing each word.

In [25]:
def delete_duplicates(document):
    return list(dict.fromkeys(document))

In [26]:
counter_dropped_duplicates = count_initializer.map(lambda word: delete_duplicates(word))
counter_dropped_duplicates.take(20)

[[('the', 1),
  ('project', 1),
  ('gutenberg', 1),
  ('ebook', 1),
  ('of', 1),
  ('frankenstein', 1),
  ('by', 1),
  ('mary', 1),
  ('wollstonecraft', 1),
  ('godwin', 1),
  ('shelley', 1)],
 [('this', 1),
  ('ebook', 1),
  ('is', 1),
  ('for', 1),
  ('the', 1),
  ('use', 1),
  ('of', 1),
  ('anyone', 1),
  ('anywhere', 1),
  ('in', 1),
  ('united', 1),
  ('states', 1),
  ('and', 1)],
 [('most', 1),
  ('other', 1),
  ('parts', 1),
  ('of', 1),
  ('the', 1),
  ('world', 1),
  ('at', 1),
  ('no', 1),
  ('cost', 1),
  ('and', 1),
  ('with', 1),
  ('almost', 1),
  ('restrictions', 1)],
 [('whatsoever', 1),
  ('you', 1),
  ('may', 1),
  ('copy', 1),
  ('it', 1),
  ('give', 1),
  ('away', 1),
  ('or', 1),
  ('reuse', 1),
  ('under', 1),
  ('the', 1),
  ('terms', 1)],
 [('of', 1),
  ('the', 1),
  ('project', 1),
  ('gutenberg', 1),
  ('license', 1),
  ('included', 1),
  ('with', 1),
  ('this', 1),
  ('ebook', 1),
  ('or', 1),
  ('online', 1),
  ('at', 1)],
 [('wwwgutenbergorg', 1),
  ('if',

In [27]:
texts_with_word = counter_dropped_duplicates.flatMap(lambda value: value).reduceByKey(lambda a, b: a + b)
texts_with_word.take(20)

[('project', 88),
 ('gutenberg', 31),
 ('ebook', 13),
 ('of', 2435),
 ('mary', 3),
 ('shelley', 3),
 ('this', 442),
 ('is', 305),
 ('use', 21),
 ('anyone', 9),
 ('anywhere', 2),
 ('in', 1127),
 ('united', 20),
 ('other', 91),
 ('world', 47),
 ('at', 318),
 ('no', 171),
 ('restrictions', 2),
 ('whatsoever', 3),
 ('may', 108)]

So we got the number of documents `df`, containing each word. Comparing the results with the previously obtained in Part 1, we make sure that the calculations are correct. 

Now to calculate IDF, we need to devide the total number of texts `D` by the number of documents with each word `df` and take 10-based logarithm.

In [28]:
import numpy as np

In [29]:
idfs_rdd = texts_with_word.map(lambda x: {x[0] : np.log10(texts_num/x[1])})
pre_dict = idfs_rdd.collect()

dict_rdd = {}
for element in pre_dict:
    dict_rdd.update(element)
    
print(dict_rdd['that'])
print(dict_rdd['online'])

0.8412473174446735
3.226406556024716


Then let's apply the same function as was used in the Part 1 (only with a different dictionary of IDF) to get the sparse TF-IDF vector.

In [30]:
def transform2(document):
    freq = {}
    tfidfs = {}
    for term in document:
        i = hash(term) % numFeatures
        freq[i] = freq.get(i, 0) + 1.0
        tfidfs[i] = freq[i]/len(document) * dict_rdd[term]
    return [document, Vectors.sparse(numFeatures, tfidfs.items())]

In [31]:
frequencyVectors2 = rdd_cleared.map(transform2)
final2 = frequencyVectors2.toDF(columns)

print('The first RDD row:')
print(frequencyVectors2.take(1))
print('')
print('Dataframe')
print(final2.show())

The first RDD row:
[[['the', 'project', 'gutenberg', 'ebook', 'of', 'frankenstein', 'by', 'mary', 'wollstonecraft', 'godwin', 'shelley'], SparseVector(7435, {285: 0.0402, 1364: 0.2468, 1482: 0.3047, 2760: 0.3047, 2941: 0.1043, 3396: 0.2125, 3737: 0.1713, 6357: 0.3047, 6747: 0.0284, 6856: 0.2151, 7280: 0.3047})]]

Dataframe
+--------------------+--------------------+
|                text|        TFIDF_vector|
+--------------------+--------------------+
|[the, project, gu...|(7435,[285,1364,1...|
|[this, ebook, is,...|(7435,[140,285,46...|
|[most, other, par...|(7435,[75,285,506...|
|[whatsoever, you,...|(7435,[632,1204,1...|
|[of, the, project...|(7435,[285,492,71...|
|[wwwgutenbergorg,...|(7435,[140,1448,3...|
|[will, have, to, ...|(7435,[285,889,12...|
|[using, this, ebook]|(7435,[1196,1364,...|
|[title, frankenst...|(7435,[5972,6856]...|
|[or, the, modern,...|(7435,[2694,3102,...|
|[author, mary, wo...|(7435,[1482,2760,...|
|[release, date, e...|(7435,[1364,1442,...|
|[most, recentl

And finally, let's bring the text indices back.

In [32]:
final2.join(rdd_with_indicies.toDF(['text', 'id']), on='text', how='left').orderBy('id').select('id', 'TFIDF_vector').show()

+---+--------------------+
| id|        TFIDF_vector|
+---+--------------------+
|  0|(7435,[285,1364,1...|
|  2|(7435,[140,285,46...|
|  2|(7435,[140,285,46...|
|  3|(7435,[75,285,506...|
|  4|(7435,[632,1204,1...|
|  5|(7435,[285,492,71...|
|  6|(7435,[140,1448,3...|
|  7|(7435,[285,889,12...|
|  8|(7435,[1196,1364,...|
| 10|(7435,[5972,6856]...|
| 11|(7435,[2694,3102,...|
| 11|(7435,[2694,3102,...|
| 13|(7435,[1482,2760,...|
| 15|(7435,[1364,1442,...|
| 16|(7435,[1412,2389,...|
| 18|(7435,[3440,4316]...|
| 20|(7435,[2501,3260,...|
| 22|(7435,[54,1267,13...|
| 23|(7435,[1909,2009,...|
| 25|(7435,[285,1364,3...|
+---+--------------------+
only showing top 20 rows

